In [1]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
import talib
import ccxt
from configparser import ConfigParser
from binance.client import Client
from binance.enums import *

In [2]:
CREDENTIALS_PATH = os.path.join(os.path.expanduser('~'), ".binance/credentials")
config = ConfigParser()
config.read(CREDENTIALS_PATH)
api_key = config.get('trade', 'API_KEY')
api_secret  = config.get('trade', 'SECRET_KEY')

In [3]:
# Initialize Variables
CANDLE_DURATION_IN_MIN = 5

RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30

CCXT_TICKER_NAME = 'DOT/USDT'
TRADING_TICKER_NAME = 'DOTUSDT'

INVESTMENT_AMOUNT_DOLLARS = 10
HOLDING_QUANTITY = 0

exchange = ccxt.binance()
binance_client = Client(api_key=api_key, api_secret=api_secret)

In [4]:
df = pd.DataFrame(binance_client.get_all_tickers())
current_price = float(df[df.symbol == "LUNAUSDT"]['price'])
current_price

93.05

In [5]:
# STEP 1: FETCH THE DATA
def fetch_data(ticker):
    global exchange
    bars,ticker_df = None, None

    try:
        bars = exchange.fetch_ohlcv(ticker, timeframe=f'{CANDLE_DURATION_IN_MIN}m', limit=100)
    except:
        print(f"Error in fetching data from the exchange:{ticker}")

    if bars is not None:
        ticker_df = pd.DataFrame(bars[:-1], columns=['at', 'open', 'high', 'low', 'close', 'vol'])
        ticker_df['Date'] = pd.to_datetime(ticker_df['at'], unit='ms')
        ticker_df['symbol'] = ticker

    return ticker_df

In [6]:
# STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
def get_trade_recommendation(ticker_df):

    macd_result = 'WAIT'
    final_result = 'WAIT'

    # BUY or SELL based on MACD crossover points and the RSI value at that point
    macd, signal, hist = talib.MACD(ticker_df['close'], fastperiod = 12, slowperiod = 26, signalperiod = 9)
    last_hist = hist.iloc[-1]
    prev_hist = hist.iloc[-2]
    if not np.isnan(prev_hist) and not np.isnan(last_hist):
        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            macd_result = 'BUY' if last_hist > 0 else 'SELL'

    if macd_result != 'WAIT':
        rsi = talib.RSI(ticker_df['close'], timeperiod = 14)
        # Consider last 3 RSI values
        last_rsi_values = rsi.iloc[-3:]

        if (last_rsi_values.min() <= RSI_OVERSOLD):
            final_result = 'BUY'
        elif (last_rsi_values.max() >= RSI_OVERBOUGHT):
            final_result = 'SELL'

    return final_result

In [7]:
# STEP 3: EXECUTE THE TRADE
def execute_trade(trade_rec_type, trading_ticker):
    global binance_client, HOLDING_QUANTITY
    order_placed = False
    side_value = SIDE_BUY if (trade_rec_type == "BUY") else SIDE_SELL
    try:
        df = pd.DataFrame(binance_client.get_all_tickers())
        df = df[df.symbol == trading_ticker]
        if True:
            current_price = float(df['price'])
            script_quantity = round(INVESTMENT_AMOUNT_DOLLARS/current_price,5) if trade_rec_type == "BUY" else HOLDING_QUANTITY
            print(f"PLACING ORDER {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}: "
                  f"{trading_ticker}, {side_value}, {current_price}, {script_quantity}, {int(time.time() * 1000)} ")
            
            ###
            order_response = binance_client.create_order(symbol=trading_ticker,
                                                         side = side_value,
                                                         type=ORDER_TYPE_LIMIT,
                                                         timeInForce = TIME_IN_FORCE_GTC,
                                                         quantity=script_quantity,
                                                         price=str(current_price))
            ###
            
            print(f"ORDER PLACED")
            HOLDING_QUANTITY = script_quantity if trade_rec_type == "BUY" else HOLDING_QUANTITY
            order_placed = True

    except:
        print(f"\nALERT!!! UNABLE TO COMPLETE ORDER")
    
    return order_placed


In [8]:
def run_bot_for_ticker(ccxt_ticker, trading_ticker):

    currently_holding = False
    while 1:
        # STEP 1: FETCH THE DATA
        ticker_data = fetch_data(ccxt_ticker)
        if ticker_data is not None:
            # STEP 2: COMPUTE THE TECHNICAL INDICATORS & APPLY THE TRADING STRATEGY
            trade_rec_type = get_trade_recommendation(ticker_data)
            print(f'{datetime.now().strftime("%d/%m/%Y %H:%M:%S")}  TRADING RECOMMENDATION: {trade_rec_type}')

            # STEP 3: EXECUTE THE TRADE
            if (trade_rec_type == 'BUY' and not currently_holding) or \
                (trade_rec_type == 'SELL' and currently_holding):
                print(f'Placing {trade_rec_type} order')
                trade_successful = execute_trade(trade_rec_type,trading_ticker)
                currently_holding = not currently_holding if trade_successful else currently_holding

            time.sleep(CANDLE_DURATION_IN_MIN*60)
        else:
            print(f'Unable to fetch ticker data - {ccxt_ticker}. Retrying!!')
            time.sleep(5)


In [9]:
run_bot_for_ticker(CCXT_TICKER_NAME,TRADING_TICKER_NAME)

22/04/2022 19:44:03  TRADING RECOMMENDATION: WAIT


KeyboardInterrupt: 